In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import gc

In [3]:
!tree .. -L 2

..
├── README.md
├── analysis
│   ├── try-lstm.ipynb
│   └── ts-sample.ipynb
└── data
    ├── example_air_passengers.csv
    └── jena_climate_2009_2016.csv

2 directories, 5 files


In [30]:
def parse(x):
    return datetime.strptime(x, '%d.%m.%Y %H:%M:%S')

df = pd.read_csv('../data/jena_climate_2009_2016.csv', parse_dates = ['Date Time'], date_parser=parse, index_col=0)

print('start date as: \t{}'.format(datetime.strftime(df.index.min(), '%Y-%m-%d')))
print('end date as: \t{}'.format(datetime.strftime(df.index.max(), '%Y-%m-%d')))
df

start date as: 	2009-01-01
end date as: 	2017-01-01


p (mbar)  T (degC)  Tpot (K)  Tdew (degC)  rh (%)  \
Date Time                                                                
2009-01-01 00:10:00    996.52     -8.02    265.40        -8.90   93.30   
2009-01-01 00:20:00    996.57     -8.41    265.01        -9.28   93.40   
2009-01-01 00:30:00    996.53     -8.51    264.91        -9.31   93.90   
2009-01-01 00:40:00    996.51     -8.31    265.12        -9.07   94.20   
2009-01-01 00:50:00    996.51     -8.27    265.15        -9.04   94.10   
...                       ...       ...       ...          ...     ...   
2016-12-31 23:20:00   1000.07     -4.05    269.10        -8.13   73.10   
2016-12-31 23:30:00    999.93     -3.35    269.81        -8.06   69.71   
2016-12-31 23:40:00    999.82     -3.16    270.01        -8.21   67.91   
2016-12-31 23:50:00    999.81     -4.23    268.94        -8.53   71.80   
2017-01-01 00:00:00    999.82     -4.82    268.36        -8.42   75.70   

                     VPmax (mbar)  VPact (mbar)  VPdef (mbar)  sh (g/kg)  \
Date Time                                                                  
2009-01-01 00:10:00          3.33          3.11          0.22       1.94   
2009-01-01 00:20:00          3.23          3.02          0.21       1.89   
2009-01-01 00:30:00          3.21          3.01          0.20       1.88   
2009-01-01 00:40:00          3.26          3.07          0.19       1.92   
2009-01-01 00:50:00          3.27          3.08          0.19       1.92   
...                           ...           ...           ...        ...   
2016-12-31 23:20:00          4.52          3.30          1.22       2.06   
2016-12-31 23:30:00          4.77          3.32          1.44       2.07   
2016-12-31 23:40:00          4.84          3.28          1.55       2.05   
2016-12-31 23:50:00          4.46          3.20          1.26       1.99   
2017-01-01 00:00:00          4.27          3.23          1.04       2.01   

                     H2OC (mmol/mol)  rho (g/m**3)  wv (m/s)  max. wv (m/s)  \
Date Time                                                                     
2009-01-01 00:10:00             3.12       1307.75      1.03           1.75   
2009-01-01 00:20:00             3.03       1309.80      0.72           1.50   
2009-01-01 00:30:00             3.02       1310.24      0.19           0.63   
2009-01-01 00:40:00             3.08       1309.19      0.34           0.50   
2009-01-01 00:50:00             3.09       1309.00      0.32           0.63   
...                              ...           ...       ...            ...   
2016-12-31 23:20:00             3.30       1292.98      0.67           1.52   
2016-12-31 23:30:00             3.32       1289.44      1.14           1.92   
2016-12-31 23:40:00             3.28       1288.39      1.08           2.00   
2016-12-31 23:50:00             3.20       1293.56      1.49           2.16   
2017-01-01 00:00:00             3.23       1296.38      1.23           1.96   

                     wd (deg)  
Date Time                      
2009-01-01 00:10:00     152.3  
2009-01-01 00:20:00     136.1  
2009-01-01 00:30:00     171.6  
2009-01-01 00:40:00     198.0  
2009-01-01 00:50:00     214.3  
...                       ...  
2016-12-31 23:20:00     240.0  
2016-12-31 23:30:00     234.3  
2016-12-31 23:40:00     215.2  
2016-12-31 23:50:00     225.8  
2017-01-01 00:00:00     184.9  

[420551 rows x 14 columns]

In [20]:
temp = df.loc[:, 'T (degC)']

In [28]:
plt.figure(figsize=(18, 8))
plt.plot(temp[:1440])   # 最早10天的数据

plt.ylabel('Temperature ($C^.$)')
plt.show()

In [48]:
df.describe().loc[['mean', 'std'], :]

p (mbar)  T (degC)  Tpot (K)  Tdew (degC)    rh (%)  VPmax (mbar)  \
mean  0.038491  0.042112  0.038693     0.070795  0.039110      0.025139   
std   0.985617  0.951522  0.949874     0.939269  0.984984      1.006484   

      VPact (mbar)  VPdef (mbar)  sh (g/kg)  H2OC (mmol/mol)  rho (g/m**3)  \
mean      0.056407     -0.008988    0.05522         0.055270     -0.034163   
std       0.996682      1.011672    0.99643         0.996077      0.940838   

       wv (m/s)  max. wv (m/s)  wd (deg)  
mean  -0.292047      -0.220088 -0.017010  
std   42.590006      29.612374  1.000788

In [49]:
tmp_df = df.iloc[:200000, :]
mean = tmp_df.mean(axis=0)
std = tmp_df.std(axis=0)

df -= mean
df /= std

df.describe().loc[['mean', 'std'], :]

p (mbar)  T (degC)  Tpot (K)  Tdew (degC)    rh (%)  VPmax (mbar)  \
mean  0.038491  0.042112  0.038693     0.070795  0.039110      0.025139   
std   0.985617  0.951522  0.949874     0.939269  0.984984      1.006484   

      VPact (mbar)  VPdef (mbar)  sh (g/kg)  H2OC (mmol/mol)  rho (g/m**3)  \
mean      0.056407     -0.008988    0.05522         0.055270     -0.034163   
std       0.996682      1.011672    0.99643         0.996077      0.940838   

       wv (m/s)  max. wv (m/s)  wd (deg)  
mean  -0.292047      -0.220088 -0.017010  
std   42.590006      29.612374  1.000788

In [46]:
del temp, tmp_df, mean, std
gc.collect()

15653

In [102]:
"""
    @ lookback : 输入数据应该包括过去多少个时间步  (5d * 24h * 6 min/h = 720)
    @ steps : 观测数据的采样频率是每小时一个数据点  (6 min/h)
    @ delay : 目标应该在未来多少个时间步之后       (24h * 6 min/h = 144)
    @ min_index / max_index : 需要抽取哪些时间步
    @ shuffle : 是打乱样本，还是按顺序抽取样本
    @ batch_size : 每个批量的样本数
    
    return
    @ samples [shape(batch_size, lookback//step, column_size)]
    @ targets  [shape(batch_size, )]
"""

def generator(data, lookback, delay, min_index, max_index, shuffle=False, batch_size=128, step=6):
    if max_index is None:
        max_index = len(data) - delay - 1
    
    i = min_index + lookback
    
    while 1:
        if shuffle:
            rows = np.random.randint(min_index+lookback, max_index, size=batch_size)
        else:
            if i + batch_size > max_index:
                i = min_index + lookback
            rows = np.arange(i, min(i + batch_size, max_index))
            i += len(rows)
            
        samples = np.zeros((
            len(rows),
            lookback // step,
            data.shape[-1]
        ))
        
        targets = np.zeros((len(rows),))
        
        for j, row in enumerate(rows):
            indices = range(rows[j]-lookback, rows[j], step)
            samples[j] = data[indices]
            targets[j] = data[rows[j] + delay][1]
            
        yield samples, targets

In [105]:
# point
train_point = [0, 200000]
val_point = [train_point[1]+1, 300000]
test_point = [val_point[1]+1, len(df)]


# parameter
lookback = 1440 
step = 6
delay = 144 
batch_size = 128

train_gen = generator(df.values, lookback=lookback, delay=delay, 
                      min_index=train_point[0], max_index=train_point[1], shuffle=True, step=step, batch_size=batch_size)

val_gen = generator(df.values, lookback=lookback,
                    delay=delay, min_index=val_point[0], max_index=val_point[1], step=step, batch_size=batch_size)

test_gen = generator(df.values, lookback=lookback,
                     delay=delay, min_index=test_point[0], max_index=None, step=step, batch_size=batch_size)

val_steps = (val_point[1] - val_point[0] - lookback) // batch_size
test_steps = (test_point[1] - test_point[0] - lookback) // batch_size

print('val step : \t{}'.format(val_steps))
print('test step : \t{}'.format(test_steps))

val step : 	769
test step : 	930


In [107]:
def evaluate_naive_method():
    batch_maes = []
    for step in range(val_steps):
        samples, targets = next(val_gen)
        preds = samples[:, -1, 1]
        mae = np.mean(np.abs(preds - targets))
        batch_maes.append(mae)
        
    print(np.mean(batch_maes))
    
evaluate_naive_method()

0.2897352486497086
